# LangChain Use-Case Example: AI API Multiplexer

This example shows how you can connect to many different AI APIs using the same code by going through LangChain.

You can run this code if you have an OpenAI API key.  If you have other API keys then you can run some of the demos that use multiple AI APIs from the same code.

## Setup code

The first code cell installs the LangChain modules and retrieves AI API credentials.  Please set up Colab secrets for any of these:

* `OPENAI_API_KEY`    -- To use the OpenAI API
* `AWS_ACCESS_KEY_ID` -- To use AWS Bedrock
* `AWS_SECRET_ACCESS_KEY`
* `AWS_REGION_NAME`

In [7]:
# Load secrets
import os
from dotenv import load_dotenv
load_dotenv()

def load_environment_variables(variable_names):
    for var_name in variable_names:
        if var_name not in os.environ:
            try:
                from google.colab import userdata
                value = userdata.get(var_name)
                if value:
                    os.environ[var_name] = value
            except ImportError:
                pass
        if var_name not in os.environ:
            raise ValueError(f"{var_name} not found. Please set it in .env file or Google Colab secrets.")
        else:
            print(f"Successfully loaded {var_name} from environment variables.")

variables_to_load = [
    "OPENAI_API_KEY",
    "AWS_ACCESS_KEY_ID",
    "AWS_SECRET_ACCESS_KEY",
    "AWS_REGION_NAME"
]

load_environment_variables(variables_to_load)

Successfully loaded OPENAI_API_KEY from environment variables.
Successfully loaded AWS_ACCESS_KEY_ID from environment variables.
Successfully loaded AWS_SECRET_ACCESS_KEY from environment variables.
Successfully loaded AWS_REGION_NAME from environment variables.


In [3]:
# Install necessary libraries
!pip install langchain langchain-community langchain-core langchain-openai

# LangChain setup: Show more about what's happening as it happens.
from langchain.globals import set_debug
set_debug(True)

## Example: Minimal AI API Request

Here's a simple OpenAI API request with LangChain.

If you do this same request [with the OpenAI API](https://platform.openai.com/docs/guides/chat-completions/overview) then you get the response in `message = completion.choices[0].message.content`.  LangChain encapsulates those details so that you can think about what you're doing instead of working to accommodate the API.

In [14]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model_name="gpt-4o")

model.invoke("Is the Sagrada Família located in Barcelona?").content

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?"
  ]
}
[llm/end] [llm:ChatOpenAI] [1.12s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. It is a large Roman Catholic basilica designed by the renowned architect Antoni Gaudí. Construction of the Sagrada Família began in 1882 and it remains unfinished to this day, although work continues with the aim of completing it in the coming years. The basilica is one of Barcelona's most iconic landmarks and a UNESCO World Heritage Site.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
         

"Yes, the Sagrada Família is located in Barcelona, Spain. It is a large Roman Catholic basilica designed by the renowned architect Antoni Gaudí. Construction of the Sagrada Família began in 1882 and it remains unfinished to this day, although work continues with the aim of completing it in the coming years. The basilica is one of Barcelona's most iconic landmarks and a UNESCO World Heritage Site."

## Example: Two Sequential Requests, Two Different Models

This example uses two different models from the same OpenAI API to handle two different parts of a larger task.

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# First LLM: Use gpt-4o for the CoT prompt to get a detailed answer
cot_prompt_template = """
Question: {question}
Please provide a detailed explanation before answering whether the statement is true or false.
"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template,
    input_variables=["question"])

# Create the first chain using OpenAI's gpt-4o model with a CoT prompt
chain_1 = ChatOpenAI(temperature=0.7, model_name="gpt-4o")

# Example question
question = "Is the Sagrada Família located in Barcelona?"

# Invoke the first chain
cot_response = chain_1.invoke(cot_prompt.format(question=question))
detailed_answer = cot_response.content
print("Detailed answer:", detailed_answer)

# Second LLM: Use gpt-4o-mini to categorize the detailed answer as 'Yes' or 'No'
yes_no_prompt_template = """
Based on the following answer, please categorize it strictly as either "Yes" or "No":
Answer: "{detailed_answer}"
"""

yes_no_prompt = PromptTemplate(
    template=yes_no_prompt_template,
    input_variables=["detailed_answer"])

# Create the second chain using OpenAI's gpt-4o-mini model for yes/no clarification
chain_2 = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# Invoke the second chain
yes_no_response = chain_2.invoke(yes_no_prompt.format(detailed_answer=detailed_answer))
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nQuestion: Is the Eiffel Tower located in Paris?\nPlease provide a detailed explanation before answering whether the statement is true or false."
  ]
}
[llm/end] [llm:ChatOpenAI] [5.10s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the statement is true; the Eiffel Tower is located in Paris.\n\nLet's delve into some details to provide a comprehensive explanation:\n\n1. **Historical Context**: The Eiffel Tower was constructed between 1887 and 1889 as the entrance arch for the 1889 Exposition Universelle (World's Fair), which was held to celebrate the 100th anniversary of the French Revolution.\n\n2. **Geographical Location**: The Eiffel Tower is situated on the Champ de Mars, a large public greenspace in the 7th arrondissement of Paris, France. The exact coordinates are approximately 48.8584° N latitude and 2.2945° E longitude.\n\n3. **Architectural Significan

# Example: Use AWS Bedrock instead of OpenAI

LangChain requests to AWS Bedrock look just like LangChain requests to OpenAI.  This example of sending a request to Claude 3.5 Sonnet looks exactly like the example above that uses OpenAI.  Even though the APIs and models have different details.

Sorting out those details could consume hours of your time up front.  And complications related to managing those details could cost you even more time and delayed progress in the future.  Shifting responsibility for those details into LangChain keeps you focused on your goals.

In [25]:
from langchain_aws import ChatBedrock

model = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20240620-v1:0",
)

model.invoke("Is the Sagrada Família located in Barcelona?.").content

[llm/start] [llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: Is the Sagrada Família located in Barcelona?."
  ]
}
[llm/end] [llm:ChatBedrock] [5.28s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Yes, the Sagrada Família is located in Barcelona, Spain. \n\nMore specifically:\n\n1. It is situated in the Eixample district of Barcelona.\n\n2. The full name of the church is the Basílica i Temple Expiatori de la Sagrada Família (Basilica and Expiatory Church of the Holy Family).\n\n3. It is one of the most famous works of the Catalan architect Antoni Gaudí.\n\n4. Construction of the Sagrada Família began in 1882 and is still ongoing, making it one of the world's longest-running architectural projects.\n\n5. It is a major landmark and tourist attraction in Barcelona, known for its unique and intricate design that combines Gothic and Art Nouveau forms.\n\n6. Despite being incomplete, the church was consecrated and declared a minor 

"Yes, the Sagrada Família is located in Barcelona, Spain. \n\nMore specifically:\n\n1. It is situated in the Eixample district of Barcelona.\n\n2. The full name of the church is the Basílica i Temple Expiatori de la Sagrada Família (Basilica and Expiatory Church of the Holy Family).\n\n3. It is one of the most famous works of the Catalan architect Antoni Gaudí.\n\n4. Construction of the Sagrada Família began in 1882 and is still ongoing, making it one of the world's longest-running architectural projects.\n\n5. It is a major landmark and tourist attraction in Barcelona, known for its unique and intricate design that combines Gothic and Art Nouveau forms.\n\n6. Despite being incomplete, the church was consecrated and declared a minor basilica by Pope Benedict XVI in 2010.\n\nThe Sagrada Família is not only located in Barcelona but is also considered one of the city's most iconic symbols and a must-see attraction for visitors to the Catalan capital."

## Example: A Sequence Of API Requests That Use Claude 3.5 Sonnet and LLama 3 8B Instruct

This code looks exactly the same as the two-step sequence above that used OpenAI.  The only thing that changed is the models.

In [26]:
# ... existing code ...

# Example: Two Sequential Requests, Two Different AWS Bedrock Models

from langchain_core.prompts import PromptTemplate
from langchain_aws import ChatBedrock

# First LLM: Use Claude 3.5 Sonnet for the CoT prompt to get a detailed answer
cot_prompt_template = """
Question: {question}
Please provide a detailed explanation before answering whether the statement is true or false.
"""

cot_prompt = PromptTemplate(
    template=cot_prompt_template,
    input_variables=["question"])

# Create the first chain using Claude 3.5 Sonnet model
chain_1 = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")

# Example question
question = "Is the Sagrada Família located in Barcelona?"

# Invoke the first chain
cot_response = chain_1.invoke(cot_prompt.format(question=question))
detailed_answer = cot_response.content
print("Detailed answer:", detailed_answer)

# Second LLM: Use Llama 2 to categorize the detailed answer as 'Yes' or 'No'
yes_no_prompt_template = """
Based on the following answer, please categorize it strictly as either "Yes" or "No":
Answer: "{detailed_answer}"
"""

yes_no_prompt = PromptTemplate(
    template=yes_no_prompt_template,
    input_variables=["detailed_answer"])

# Create the second chain using Meta's Llama 2 model for yes/no clarification
chain_2 = ChatBedrock(model_id="meta.llama3-8b-instruct-v1:0")

# Invoke the second chain
yes_no_response = chain_2.invoke(yes_no_prompt.format(detailed_answer=detailed_answer))
yes_no_answer = yes_no_response.content.strip()
print("Yes/No Answer:", yes_no_answer)

[llm/start] [llm:ChatBedrock] Entering LLM run with input:
{
  "prompts": [
    "Human: \nQuestion: Is the Sagrada Família located in Barcelona?\nPlease provide a detailed explanation before answering whether the statement is true or false."
  ]
}
[llm/end] [llm:ChatBedrock] [6.18s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Let's explore the details about the Sagrada Família:\n\nThe Sagrada Família is a large and iconic basilica located in Barcelona, Spain. It is one of the most famous works of the Catalan architect Antoni Gaudí, who devoted much of his life to this project. Construction began in 1882 and is still ongoing, making it one of the longest-running architectural projects in the world.\n\nKey points about the Sagrada Família:\n\n1. Location: It is situated in the Eixample district of Barcelona, Catalonia, Spain.\n\n2. Design: The basilica features a unique blend of Gothic and Art Nouveau styles, with intricate facades and towering spires